# Manually add a MISP event and attribute

This notebook shows you how to manually add a MISP event and one attribute, all with the use of PyMISP.

# Configure PyMISP

The configuration of PyMISP is done via the **keys.py** file. 

This file needs to contain at least the **URL** of the MISP server (misp_url) and an **authentication key** (misp_key). You can have multiple keys per user, stored in separate files. Make sure you restrict access to the keys as they give exactly the same permissions as the associated user.

Instead of a keys file you can also supply the configuration directly via variables.

First output what is in our keys file stored in the demo path.

In [ ]:
cat ~/demo/PyMISP/keys.py

Then initialise PyMISP and disable the "Unverified HTTPS request" warnings.

In [ ]:
import urllib3
from pymisp import PyMISP, MISPEvent
from keys import misp_url, misp_key, misp_verifycert

if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = PyMISP(misp_url, misp_key, misp_verifycert)
    
print("I will use the server {}".format(misp_url))

# Create a MISP event

## Basic event elements

You need to supply at least an event **title** (info). Optionally add the distribution, threat level and analysis state. These fields expect a numerical format. 

**Distribution**
- 0: Your organization only
- 1: This community-only
- 2: Connected communities
- 3: All communities

**Threat level**
- 1: Low (mass malware)
- 2: Medium (APT)
- 3: High (0-day)
- 4: Undefined

**Analysis**
- 0: Initial
- 1: Ongoing
- 2: Completed

In [ ]:
event = MISPEvent()

event.info = "Demo event for MISP Tip of the Week"
event.distribution = 3
event.threat_level_id = 4
event.analysis = 1

## Add a date

In [ ]:
event.set_date("2022-06-10")

## Add a tag

In [ ]:
event.add_tag("tlp:amber")

## Output the event object

Before we continue, let's first print what's in the event object.

In [ ]:
print(event.to_json())

## Create event in MISP

Send the request to the server

In [ ]:
result = misp.add_event(event, pythonify=True)
print("Created event ID {} with UUID {}".format(result.id, result.uuid))

# Add attributes

Next we add one attribute. 

Make sure you set the **type** and the **value**, and optionally the **to_ids** flag. It's also a good idea to add contextualisation to the attribute.

In [ ]:
from pymisp import MISPAttribute

attribute = MISPAttribute()
attribute.type = "ip-dst"
attribute.value = "8.8.4.4"
attribute.to_ids = False
attribute.add_tag("tlp:white")
attribute.comment = "There is an initial request to this IP"

## Output the attribute event

Before we continue, let's first print what's in the attribute object.

In [ ]:
print(attribute.to_json())

## Add to event

And now add the attribute to the event

In [ ]:
result_attr = misp.add_attribute(result.id, attribute, pythonify=True)
print("Added attribute {}".format(result_attr.uuid))

# Get the event from MISP

Now, as a verification step we get the event from MISP and output the result.

In [ ]:
verify_event = misp.get_event(result.id, pythonify=True)
print(verify_event.to_json())